In [1]:
"""
#CURRENT VERSION 
A simple test of Ray
This example uses placement_group API to spread work around
"""
import random
import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self, actor_id) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()
        self.actor_id = actor_id
        
    def ping(self):
        print(f"{self.actor_id} {self.pid} {self.hostname} {self.ip} {time.time()} - ping")
        time.sleep(random.randint(1,3))
        return f"{self.actor_id}"

@ray.remote
def main():
# Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")
# Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    time.sleep(1)
    
# Ping-Pong test
    messages = [actors[a].ping.remote() for a in actors]
    time.sleep(1)
    for _ in range(10):
        new_messages, messages = ray.wait(messages, num_returns=1)
        for ray_message_id in new_messages:
            pong = ray.get(ray_message_id)
            print(pong, "- pong")
            check = actors[pong].ping.remote()
            time.sleep(1)
            messages.append(check)

actors = {
    "actor1" : Actor.remote(actor_id="actor1"),
    "actor2" : Actor.remote(actor_id="actor2"),
    "actor3" : Actor.remote(actor_id="actor3"),
    "actor4" : Actor.remote(actor_id="actor4"),
    "actor5" : Actor.remote(actor_id="actor5")
}

print(actors)
if __name__ == "__main__":
    main.remote()

2021-07-10 17:26:40,493	INFO services.py:1317 -- View the Ray dashboard at http://127.0.0.1:8265


{'actor1': Actor(Actor,9eb69dc269d8b410e9f55c0801000000), 'actor2': Actor(Actor,f72a5f235b5389ad6b3fc1f901000000), 'actor3': Actor(Actor,57076b4dbfb604d5978d896701000000), 'actor4': Actor(Actor,805b8c0e2a62c5e2f78f416101000000), 'actor5': Actor(Actor,cdae3f8e9d97fd70a7f40de501000000)}
(pid=8740) Found 5 Worker nodes in the Ray Cluster:
(pid=8740) Setting up 5 Actors...
(pid=8728) actor2 8728 Anthonys-Air 192.168.1.214 1625963210.828779 - ping
(pid=8727) actor3 8727 Anthonys-Air 192.168.1.214 1625963210.8291461 - ping
(pid=8726) actor1 8726 Anthonys-Air 192.168.1.214 1625963210.8273568 - ping
(pid=8725) actor4 8725 Anthonys-Air 192.168.1.214 1625963210.829812 - ping
(pid=8742) actor5 8742 Anthonys-Air 192.168.1.214 1625963210.829812 - ping
(pid=8740) actor2 - pong
(pid=8728) actor2 8728 Anthonys-Air 192.168.1.214 1625963212.8407109 - ping
(pid=8740) actor1 - pong
(pid=8726) actor1 8726 Anthonys-Air 192.168.1.214 1625963213.846362 - ping
(pid=8740) actor5 - pong
(pid=8742) actor5 8742 An

In [ ]:
# VERSION TWO
"""
A simple test of Ray
This example uses placement_group API to spread work around
"""

import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()

    def ping(self):
        
        print(f"{self.pid} {self.hostname} {self.ip} - ping")
        time.sleep(1)
        return f"{self.pid} {self.hostname} {self.ip} - pong"

@ray.remote
def main():
   
    # Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")

    # Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    actor = []
    for a in actors:
        node_ip_str = f"node:{ray._private.services.get_node_ip_address()}"
        actor.append(Actor.remote())
        time.sleep(1)

    # Ping-Pong test
    for _ in range(2):
        for a in actors:
            time.sleep(1)
            messages = a.ping.remote()
            time.sleep(1)
            print(f"Received back message {ray.get(messages)} \n")
        

            
actors = [Actor.remote() for _ in range(5)]
print(actors)

if __name__ == "__main__":
    main.remote()

In [ ]:
# VERSION THREE
"""
A simple test of Ray
This example uses placement_group API to spread work around
"""

import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()

    def ping(self):
        
        print(f"{self.pid} {self.hostname} {self.ip} - ping")
        time.sleep(1)
        return f"{self.pid} {self.hostname} {self.ip} - pong"

@ray.remote
def main():
   
    # Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")

    # Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    actor = []
    for a in actors:
        node_ip_str = f"node:{ray._private.services.get_node_ip_address()}"
        actor.append(Actor.remote())
        time.sleep(1)

    # Ping-Pong test
    for _ in range(2):
        messages = [a.ping.remote() for a in actors]
        for msg in ray.get(messages):
            time.sleep(1)
            print(f"Received back message {msg}")
            time.sleep(1)

            
actors = [Actor.remote() for _ in range(5)]
print(actors)

if __name__ == "__main__":
    main.remote()